In [1]:
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch

In [4]:
import pandas

In [28]:
clues = load_dataset("gips-mai/all_clues_enc", split='train')
len_countries = len(clues['country_one_hot_enc'][6][0])
len_countries

Generating train split:   0%|          | 0/4120 [00:00<?, ? examples/s]

221

In [29]:
one_hot_encoding = [torch.zeros(len_countries) for i in range(len(clues))]

for i, c in enumerate(clues['country_one_hot_enc']):
    for enc in c:
        if len(enc) > 0:
            one_hot_encoding[i] += torch.Tensor(enc)
        else:
            print(c)
    one_hot_encoding[i] = list(one_hot_encoding[i].numpy().astype('int'))


[[]]


In [30]:
csv = clues.to_pandas()
csv['country_one_hot_enc'] = one_hot_encoding

In [31]:
csv

,clue_id,text,iso2,clue_type,iso3,country_one_hot_enc,encoding
0,0,"In Street View, if the sun is clearly in the N...",[],0,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[[-0.06175443, 0.11750414, -0.07026248, -0.11..."
1,1,If the tip of a shadow is pointing south then ...,[],0,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[[-0.050538097, 0.119959146, -0.06888637, -0...."
2,2,The direction that the shadow’s tip points gen...,[],0,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[[-0.045950864, 0.11772313, -0.057295177, -0...."
3,3,If the sun is in the Northern Hemisphere then ...,[],0,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[[-0.063013576, 0.12525813, -0.043623682, -0...."
4,4,On some occasions the sun will appear overhead...,[],0,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[[-0.088886335, 0.12585206, -0.037870497, -0...."
...,...,...,...,...,...,...,...
4115,4115,Subtropical with rapid changes in temperature ...,['PY'],1,['PRY'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[[-0.03873987, 0.13176878, -0.008881576, -0.0..."
4116,4116,The weather in Peru varies according to area –...,['PE'],1,['PER'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[[-0.04640855, 0.16804248, -0.022369253, -0.1..."
4117,4117,"Tropical and humid, cooled by the northeast tr...",['SR'],1,['SUR'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[[-0.03169726, 0.137604, 0.004715935, -0.0779..."
4118,4118,Uruguay has an exceptionally fine temperate cl...,['UY'],1,['URY'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[[-0.03657007, 0.15742064, -0.006655157, -0.0..."


In [42]:
csv.to_csv('../data/all_clues_batchable.csv', index=False)

In [24]:
from typing import Any
from torch.utils.data import Dataset

class PandasDataset(Dataset):
    def __init__(self, df) -> None:
        self.dataframe = df

    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, index) -> Any:
        return self.dataframe.iloc[index]

In [35]:
data_loader = DataLoader(csv, batch_size=2, shuffle=False)

In [36]:
for batch in data_loader:
    print(batch)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [19]:
# Description Emb., Img Emb., Country Hot Encoding, Cell Target, Coordinate Target (Lat, Lon)
n = 124
descripts = [torch.randn(716).numpy() for i in range(n)]
img_embeds = [torch.randn(716).numpy() for i in range(n)]

country_encodings = []
for i in range(n):
    enc = torch.zeros(221).numpy()
    enc[torch.randint(0, 221, (1, ))[0]] = 1
    country_encodings.append(enc)

cell_targets = [torch.randint(0, 10000, (1, ))[0].numpy() for i in range(n)]

coordinate_targets = []
for i in range(n):
    enc = torch.randn(2).numpy()
    enc[0] *= 180
    enc[1] *= 90
    coordinate_targets.append(enc)


In [22]:
df = pandas.DataFrame(list(zip(descripts, img_embeds, country_encodings, cell_targets, coordinate_targets)),
               columns =['descriptions', 'img_emb', "country_enc", "cell_target", "coordinate_target"])


In [25]:
pd_dataset = PandasDataset(df)

In [27]:
for batch in pd_dataset:
    print(batch)

descriptions         [-0.2736465, -0.19129403, -1.3531156, 0.718762...
img_emb              [0.8836805, 0.87347895, 1.8504995, -0.64792633...
country_enc          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
cell_target                                                       3415
coordinate_target                              [-258.26294, 19.450367]
Name: 0, dtype: object
descriptions         [0.38016614, 1.3823583, 0.4324131, 0.76037997,...
img_emb              [-0.28293484, -0.44345924, 0.66806614, 1.21766...
country_enc          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
cell_target                                                       6670
coordinate_target                              [197.96661, -167.18463]
Name: 1, dtype: object
descriptions         [1.7503697, -1.516952, -1.2567549, 2.2620726, ...
img_emb              [2.228789, 0.3335889, 0.6212683, 0.8596163, -0...
country_enc          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
cell_target                    

In [ ]:
from pathlib import Path
import sys
sys.path.append(Path.cwd().parent)

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from model.head.geolocation_head import MLPCentroid, HybridHeadCentroid

from datasets import load_dataset

### HYPER PARAMETERS ###
lr = 0.001
alpha = 0.75
use_tanh=True
scale_tanh=1.2
### HYPER PARAMETERS ###

device = 'cuda'

labels = pd.read_csv('path/to/labels.csv')
country_encoding = pd.read_csv('path/to/country/encoding.csv')

clues = pd.read_pickle("path/to/clues.pkl")
clue_embedding_size:int = 512

encoded_descriptions = pd.read_csv('path/to/encoded/descriptions.csv')
text_embedding_size:int = 512

clip_embedding_size:int = 768

previous_stage_output = text_embedding_size+clip_embedding_size+clue_embedding_size
geohead = MLPCentroid(initial_dim=previous_stage_output, hidden_dim=[previous_stage_output, 1024, 512])
hybrid_head_centroid = HybridHeadCentroid(final_dim=11398, path='../data/quadtree/quadtree_10_1000.csv', use_tanh=use_tanh, scale_tanh=scale_tanh)

optimizer = optim.Adam(list(geohead.parameters()))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5)

cell_loss = nn.CrossEntropyLoss()
coordinate_loss = nn.MSELoss()
pseudo_label_loss = nn.MSELoss()

clues = load_dataset("gips-mai/all_clues_enc")
descriptions = load_dataset("gips-mai/enc_descr")
data_loader = torch.utils.data.DataLoader(descriptions, batch_size=32, shuffle=True)

country_losses = []
geo_losses = []
for epoch in range(10):
    for batch in data_loader:
        imgs, descriptions, country_target, cell_target, coordinate_target = batch
        imgs, descriptions, country_target, cell_target, coordinate_target = imgs.to(device), descriptions.to(device), country_target.to(device), cell_target.to(device), coordinate_target.to(device)

        optimizer.zero_grad()

        aggregated_input = torch.cat(imgs, dim=1)

        intermediate = geohead.forward(aggregated_input)
        prediction = hybrid_head_centroid.forward(intermediate, cell_target)

        total_loss = cell_loss.apply(prediction['label'], cell_target) + coordinate_loss.apply(prediction['gps'], coordinate_target)

        total_loss.backward()
        optimizer.step()

    scheduler.step()

    # Print the loss at each epoch
    print(f"Epoch {epoch+1}, Loss: {total_loss.item():.4f}")